In [ ]:
import os
import getpass
from transformers import AutoTokenizer, AutoModel
import torch
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.tools import Tool
from langchain_groq import ChatGroq
from langchain.chains import ConversationalRetrievalChain
from langchain.agents import Tool, initialize_agent, AgentType
from langchain_core.messages import HumanMessage

# Set the API key for Groqcloud
if not os.environ.get("GROQ_API_KEY"):
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter API key for Groq: ")

Enter API key for Groq: ··········


In [ ]:
# Hugging Face model for embeddings
model_name = "sentence-transformers/all-MiniLM-L6-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
hf_model = AutoModel.from_pretrained(model_name)

# Embedding function
def generate_embeddings(texts):
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        embeddings = hf_model(**inputs).last_hidden_state.mean(dim=1)  # Mean pooling
    return embeddings

# Load multiple PDF documents
pdf_files = [
    "/content/Calculus.pdf",
    "/content/Thomas' Calculus--13th Edition.pdf",
    "/content/Linear Algebra And Its Applications   - David C Lay.pdf",
    "/content/Walpole Stats.pdf" # Add more PDFs as needed
]

documents = []
for pdf_file in pdf_files:
    loader = PyPDFLoader(pdf_file)
    documents.extend(loader.load_and_split())

# Convert documents to text for embeddings
document_texts = [doc.page_content for doc in documents]

# Use Hugging Face Embeddings (LangChain-compatible wrapper)
hf_embeddings = HuggingFaceEmbeddings(model_name=model_name)

# Create Chroma vectorstore (you can also store embeddings to disk to avoid memory issues)
vectorstore = Chroma.from_texts(document_texts, embedding=hf_embeddings)

# Set up retriever
retriever = vectorstore.as_retriever()




/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
<ipython-input-2-8f5c690a4d5b>:30: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hf_embeddings = HuggingFaceEmbeddings(model_name=model_name)


In [ ]:
# Define Groq model for LLM
llm = ChatGroq(model="llama3-8b-8192")

# Define retrieval-based QA chain
chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True  # Include source documents
)

# Ask a question and print response
response = chain.invoke({"query": "What is the derivative of x^2 + 3x + 2?"})

# Access the answer and source documents
print("Answer:", response["result"])
print("Source Documents:", response["source_documents"])

# Define the math tool function
def math_tool_function(query: str) -> str:
    try:
        return str(eval(query))
    except Exception as e:
        return f"Error: {e}"

# Define the math tool
math_tool = Tool(
    name="MathTool",
    func=math_tool_function,
    description="Tool for evaluating mathematical expressions."
)

Answer: To find the derivative of x^2 + 3x + 2, we can use the power rule of differentiation, which states that if f(x) = x^n, then f'(x) = nx^(n-1).

So, for the first term x^2, we have:

d(x^2)/dx = 2x^(2-1) = 2x

For the second term 3x, we have:

d(3x)/dx = 3

For the third term 2, we have:

d(2)/dx = 0 (since the derivative of a constant is 0)

Therefore, the derivative of x^2 + 3x + 2 is:

2x + 3

This is the same result that would be obtained by using the definition of a derivative and plugging in the function x^2 + 3x + 2.
Source Documents: [Document(metadata={}, page_content='Chapter 3 : Derivatives Section 3.1 : The Definition of the Derivative\nFirstplug the functioninto the definition ofthe derivative.\nf′ (x) = lim\nh→0\nf (x + h) − f (x)\nh\n= lim\nh→0\n2(x + h)2 − 16 (x + h) + 35 −\n(\n2x2 − 16x + 35\n)\nh\nBe careful and make sure that you properly deal with parenthesis when doing the subtract-\ning.\nNow,weknowfromthepreviouschapterthatwecan’tjustplugin h = 0 sincethisw

In [ ]:
# Create a conversational retrieval chain
retrieval_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,  # Your language model (e.g., Groq)
    retriever=retriever,  # Your retriever (e.g., Chroma retriever)
    return_source_documents=True  # This should ensure source docs are included
)


# Initialize an agent with the math tool and retrieval chain
agent = initialize_agent(
    tools=[math_tool],  # Include the math tool
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,  # Select an agent type
    llm=llm,  # Your Groq model
    verbose=True  # Optional: Set to True for debugging
)



<ipython-input-4-10f03d10a13a>:10: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. See LangGraph documentation for more details: https://langchain-ai.github.io/langgraph/. Refer here for its pre-built ReAct agent: https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/
  agent = initialize_agent(


In [ ]:
# Define an empty chat history for the initial query
chat_history = []

# Step 1: Retrieve relevant documents for the first query
documents_1 = retriever.get_relevant_documents("What is the derivative of x^2 + 3x + 2?")
print("Retrieved Documents for Response 1:", documents_1)

# Step 2: Ask the first question and update chat history, passing the documents
response_1 = agent.invoke({
    "input": "What is the derivative of x^2 + 3x + 2?",
    "chat_history": chat_history,
    "documents": documents_1  # Pass the retrieved documents as context
})

# Print the response to inspect its structure and the source documents
print("Response 1:", response_1)

# Check the structure of the response
if isinstance(response_1, dict) and "result" in response_1:
    # If 'result' key exists, process as usual
    chat_history.append({"role": "user", "content": "What is the derivative of x^2 + 3x + 2?"})
    chat_history.append({"role": "assistant", "content": response_1["result"]})

    # Print the source documents if available
    if 'source_documents' in response_1:
        print("Source Documents for Response 1:", response_1['source_documents'])
    else:
        print("No source documents available for Response 1.")

# Step 3: Ask the second question using the updated chat history
response_2 = agent.invoke({
    "input": "Can you explain the result?",
    "chat_history": chat_history,
    "documents": documents_1  # Pass the same documents if needed
})

# Print the second response for debugging and the source documents
print("Response 2:", response_2)

# Handle response_2 similarly as above
if isinstance(response_2, dict) and "result" in response_2:
    chat_history.append({"role": "user", "content": "Can you explain the result?"})
    chat_history.append({"role": "assistant", "content": response_2["result"]})

    # Print the source documents if available
    if "source_documents" in response_2:
        print("Source Documents for Response 2:", response_2["source_documents"])

# Step 4: Ask the third question related to Linear Algebra using the content of the PDFs
documents_3 = retriever.get_relevant_documents("Solve the system of linear equations: x + y = 5, 2x - y = 3")
response_3 = agent.invoke({
    "input": "Solve the system of linear equations: x + y = 5, 2x - y = 3",
    "chat_history": chat_history,
    "documents": documents_3  # Pass the retrieved documents for this question
})

# Print the third response for debugging and the source documents
print("Response 3:", response_3)

# Handle response_3 similarly as above
if isinstance(response_3, dict) and "result" in response_3:
    chat_history.append({"role": "user", "content": "Solve the system of linear equations: x + y = 5, 2x - y = 3"})
    chat_history.append({"role": "assistant", "content": response_3["result"]})

    # Print the source documents if available
    if "source_documents" in response_3:
        print("Source Documents for Response 3:", response_3["source_documents"])

# Step 5: Ask a follow-up question related to the solution
response_4 = agent.invoke({
    "input": "Can you explain the solution and how it was derived from the documents?",
    "chat_history": chat_history,
    "documents": documents_3  # Pass the same documents if needed
})

# Print the response and source documents
print("Response 4:", response_4)

# Check for source documents in response_4
if isinstance(response_4, dict) and "result" in response_4:
    # Print the source documents if available
    if "source_documents" in response_4:
        print("Source Documents for Response 4:", response_4["source_documents"])


Retrieved Documents for Response 1: [Document(metadata={}, page_content='Chapter 3 : Derivatives Section 3.1 : The Definition of the Derivative\nFirstplug the functioninto the definition ofthe derivative.\nf′ (x) = lim\nh→0\nf (x + h) − f (x)\nh\n= lim\nh→0\n2(x + h)2 − 16 (x + h) + 35 −\n(\n2x2 − 16x + 35\n)\nh\nBe careful and make sure that you properly deal with parenthesis when doing the subtract-\ning.\nNow,weknowfromthepreviouschapterthatwecan’tjustplugin h = 0 sincethiswillgiveus\nadivisionbyzeroerror. So,wearegoingtohavetodosomework. Inthiscasethatmeans\nmultiplyingeverythingoutanddistributingtheminussignthroughonthesecondterm. Doing\nthisgives,\nf′ (x) = lim\nh→0\n2x2 + 4xh + 2h2 − 16x − 16h + 35 − 2x2 + 16x − 35\nh\n= lim\nh→0\n4xh + 2h2 − 16h\nh\nNotice that every term in the numerator that didn’t have anh in it canceled out and we can\nnow factor anh out of the numerator which will cancel against theh in the denominator.\nAfterthat we can computethe limit.\nf′ (x) = lim\nh→

In [12]:
# Save Hugging Face model and tokenizer to disk
model_name = "sentence-transformers/all-MiniLM-L6-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
hf_model = AutoModel.from_pretrained(model_name)

# Save models
tokenizer.save_pretrained("/content/all-MiniLM-L6-v2-tokenizer")
hf_model.save_pretrained("/content/all-MiniLM-L6-v2-model")


In [15]:
from langchain.vectorstores import Chroma

# Save the Chroma vector store to disk
vectorstore = Chroma.from_texts(document_texts, embedding=hf_embeddings, persist_directory="/content/vectorstore")
# Pass persist_directory during initialization

# The persist() method is called automatically when persist_directory is provided during initialization.
# You no longer need to call vectorstore.persist() separately.

In [16]:
# Upload the files manually or programmatically if necessary
pdf_files = [
    "/content/Calculus.pdf",
    "/content/Thomas' Calculus--13th Edition.pdf",
    "/content/Linear Algebra And Its Applications - David C Lay.pdf",
    "/content/Walpole Stats.pdf"  # Ensure these paths are correct in your environment
]


In [21]:
# Save the tokenizer and model locally
tokenizer.save_pretrained("./saved_tokenizer")
hf_model.save_pretrained("./saved_model")


In [22]:
# Save the tokenizer and model locally in Colab
model_save_path = "/content/saved_model"
tokenizer_save_path = "/content/saved_tokenizer"

tokenizer.save_pretrained(tokenizer_save_path)
hf_model.save_pretrained(model_save_path)


In [25]:
# Save the vectorstore in Colab
vectorstore_save_path = "/content/vectorstore"

# The persist() method is no longer needed. The vectorstore is already persisted during creation.
# Instead of calling persist, this line is just for information
print(f"Vectorstore persisted to: {vectorstore_save_path}")

# You should have already persisted it using the following in a previous cell:
# vectorstore = Chroma.from_texts(document_texts, embedding=hf_embeddings, persist_directory=vectorstore_save_path)

Vectorstore persisted to: /content/vectorstore


In [27]:
from google.colab import files
import shutil

# Zip the model and tokenizer folders to download
shutil.make_archive("/content/saved_model_and_tokenizer", 'zip', "/content")

# Download the zip file to your local machine
files.download("/content/saved_model_and_tokenizer.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [28]:
# Zip the vectorstore folder to download
shutil.make_archive("/content/vectorstore", 'zip', "/content/vectorstore")

# Download the zip file to your local machine
files.download("/content/vectorstore.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>